<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax (0, 1) Scaler Informer](#3-minmax-scaler-0-1-informer)
- [4. MinMax (0, 1) Scaler PatchTST](#4-minmax-scaler-0-1-patchtst)
- [5. MinMax (0, 5) Scaler Informer](#5-minmax-scaler-0-5-informer)
- [6. MinMax (0, 5) Scaler PatchTST](#6-minmax-scaler-0-5-patchtst)

</details>


Here we perform a check on DE dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [2]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [3]:
cuda_device = "0"

# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.00001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Set the best learning rate based on pred_len
            if pred_len == "24":
                lr = 0.00001
            elif pred_len in ["96", "168"]:
                lr = 0.0001

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 50 \
              --patience 5 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rat

In [5]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2203  0.4694  0.2935  0.4299
              2         24        0.2171  0.4659  0.2862  0.4267
              1         96        0.3869  0.6220  0.4299  0.5696
              2         96        0.3770  0.6140  0.4187  0.5622
              1         168       0.4195  0.6477  0.4588  0.5932
              2         168       0.4416  0.6645  0.4715  0.6086
MAE           1         24        0.2123  0.4607  0.2703  0.4220
              2         24        0.2143  0.4629  0.2699  0.4240
              1         96        0.4036  0.6353  0.3978  0.5816
              2         96        0.3973  0.6303  0.3909  0.5771
              1         168       0.4300  0.6558  0.4283  0.6006
              2         168       0.4308  0.6564  0.4311  0.6012

In [6]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1680994.875  1296.5319   861.9956  0.0911
              2         24        1580196.750  1257.0587   835.5945  0.0883
              1         96        3797896.500  1948.8192  1355.7372  0.1371
              2         96        3531860.500  1879.3245  1300.5291  0.1323
              1         168       4956448.000  2226.3081  1516.3369  0.1568
              2         168       5682674.500  2383.8362  1595.9253  0.1679
MAE           1         24        1440519.500  1200.2164   759.9636  0.0843
              2         24        1422171.750  1192.5485   752.7681  0.0838
              1         96        3235874.000  1798.8535  1184.2261  0.1266
              2         96        3134767.500  1770.5275  1156.8478  0.1246
              1         168       4295732.000  2072.6147  1351.1304  0.1460
              2         168       4343271.000  2084.0515  1365.0415  0.1468

In [7]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2133  0.4618  0.2701  0.4230
         MSE            0.2187  0.4676  0.2899  0.4283
96       MAE            0.4004  0.6328  0.3944  0.5794
         MSE            0.3820  0.6180  0.4243  0.5659
168      MAE            0.4304  0.6561  0.4297  0.6009
         MSE            0.4305  0.6561  0.4652  0.6009

In [8]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.431346e+06  1196.3824   756.3658  0.0841
         MSE            1.630596e+06  1276.7953   848.7950  0.0897
96       MAE            3.185321e+06  1784.6905  1170.5369  0.1256
         MSE            3.664878e+06  1914.0718  1328.1331  0.1347
168      MAE            4.319502e+06  2078.3331  1358.0859  0.1464
         MSE            5.319561e+06  2305.0721  1556.1311  0.1624

# 2. Standard Scaler PatchTST

In [9]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.00001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 50 \
              --patience 5 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_r

: 

In [10]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2163  0.4650  0.2981  0.4259
              2         24        0.2257  0.4751  0.3017  0.4351
              1         96        0.3643  0.6036  0.3992  0.5526
              2         96        0.3615  0.6012  0.3974  0.5505
              1         168       0.3864  0.6216  0.4178  0.5693
              2         168       0.3866  0.6218  0.4179  0.5695
RMSE          1         24        0.2188  0.4678  0.3057  0.4284
              2         24        0.2228  0.4720  0.2965  0.4323
              1         96        0.3633  0.6027  0.3975  0.5519
              2         96        0.3603  0.6002  0.3956  0.5496
              1         168       0.3849  0.6204  0.4157  0.5682
              2         168       0.3846  0.6202  0.4157  0.5680
MAE           1         24        0.2347  0.4844  0.3008  0.4436
              2         24        0.2179  0.4668  0.2705  0.4275
              1         96        0.3723  0.6102  0.3903  0.5587
              2         96        0.3720  0.6099  0.3905  0.5585
              1         168       0.3949  0.6284  0.4102  0.5755
              2         168       0.3947  0.6283  0.4102  0.5754

In [11]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1452499.000  1205.1967   853.6146  0.0847
              2         24        1513613.875  1230.2902   866.8263  0.0865
              1         96        3107539.250  1762.8214  1208.1465  0.1241
              2         96        3066246.250  1751.0701  1197.7284  0.1232
              1         168       3617150.250  1901.8807  1296.1769  0.1340
              2         168       3511273.500  1873.8392  1283.4158  0.1320
RMSE          1         24        1550383.250  1245.1439   886.2724  0.0875
              2         24        1480387.625  1216.7118   848.0317  0.0855
              1         96        3056700.250  1748.3422  1198.0056  0.1230
              2         96        3011115.000  1735.2565  1186.3693  0.1221
              1         168       3557823.250  1886.2194  1284.5880  0.1329
              2         168       3453479.000  1858.3539  1271.8232  0.1309
MAE           1         24        1737740.625  1318.2339   870.5349  0.0926
              2         24        1284505.750  1133.3604   733.4088  0.0796
              1         96        2971120.250  1723.6937  1153.3737  0.1213
              2         96        2962697.000  1721.2487  1151.7640  0.1211
              1         168       3482605.250  1866.1740  1247.3425  0.1315
              2         168       3453181.500  1858.2738  1243.2384  0.1309

In [12]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2263  0.4756  0.2857  0.4356
         MSE            0.2210  0.4701  0.2999  0.4305
         RMSE           0.2208  0.4699  0.3011  0.4303
96       MAE            0.3722  0.6100  0.3904  0.5586
         MSE            0.3629  0.6024  0.3983  0.5516
         RMSE           0.3618  0.6015  0.3965  0.5507
168      MAE            0.3948  0.6283  0.4102  0.5755
         MSE            0.3865  0.6217  0.4178  0.5694
         RMSE           0.3848  0.6203  0.4157  0.5681

In [13]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.511123e+06  1225.7971   801.9718  0.0861
         MSE            1.483056e+06  1217.7434   860.2204  0.0856
         RMSE           1.515385e+06  1230.9279   867.1521  0.0865
96       MAE            2.966909e+06  1722.4712  1152.5688  0.1212
         MSE            3.086893e+06  1756.9457  1202.9374  0.1236
         RMSE           3.033908e+06  1741.7993  1192.1874  0.1226
168      MAE            3.467893e+06  1862.2239  1245.2905  0.1312
         MSE            3.564212e+06  1887.8600  1289.7963  0.1330
         RMSE           3.505651e+06  1872.2866  1278.2056  0.1319

In [14]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled_IT')

# 3. MinMax Scaler (0, 1) Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [3]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [4]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.00001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0

In [5]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_IT.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_IT.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0107  0.1035  0.0632  0.3912
              2         24        0.0109  0.1043  0.0648  0.3943
              1         96        0.0188  0.1372  0.0888  0.5187
              2         96        0.0184  0.1358  0.0881  0.5135
              1         168       0.0202  0.1421  0.0942  0.5377
              2         168       0.0205  0.1433  0.0958  0.5421
RMSE          1         24        0.0109  0.1043  0.0635  0.3943
              2         24        0.0112  0.1060  0.0651  0.4005
              1         96        0.0188  0.1370  0.0885  0.5180
              2         96        0.0183  0.1353  0.0879  0.5118
              1         168       0.0205  0.1433  0.0946  0.5422
              2         168       0.0204  0.1429  0.0945  0.5408
MAE           1         24        0.0115  0.1072  0.0629  0.4050
              2         24        0.0111  0.1051  0.0611  0.3973
              1         96        0.0200  0.1414  0.0888  0.5345
              2         96        0.0203  0.1425  0.0888  0.5387
              1         168       0.0230  0.1516  0.0949  0.5738
              2         168       0.0225  0.1500  0.0947  0.5674

In [6]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1526122.125  1235.3632   813.0915  0.0868
              2         24        1720168.500  1311.5520   849.8506  0.0922
              1         96        3564554.250  1888.0027  1239.8208  0.1329
              2         96        3233782.000  1798.2720  1197.2759  0.1266
              1         168       4201466.000  2049.7478  1339.4785  0.1444
              2         168       4258257.500  2063.5547  1360.0095  0.1454
RMSE          1         24        1534866.750  1238.8973   812.3772  0.0871
              2         24        1773736.375  1331.8169   851.7638  0.0936
              1         96        3390077.000  1841.2162  1212.3903  0.1296
              2         96        3278264.250  1810.5978  1197.8483  0.1274
              1         168       4204012.500  2050.3689  1340.2507  0.1444
              2         168       4257367.000  2063.3389  1350.4883  0.1453
MAE           1         24        1691501.875  1300.5775   802.2434  0.0914
              2         24        1389721.625  1178.8645   748.0410  0.0828
              1         96        3605560.250  1898.8313  1205.3231  0.1336
              2         96        3148950.000  1774.5281  1153.3165  0.1249
              1         168       3959525.750  1989.8557  1283.2562  0.1402
              2         168       4440883.000  2107.3403  1338.4319  0.1484

In [7]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0113  0.1061  0.0620  0.4011
         MSE            0.0108  0.1039  0.0640  0.3928
         RMSE           0.0111  0.1052  0.0643  0.3974
96       MAE            0.0201  0.1419  0.0888  0.5366
         MSE            0.0186  0.1365  0.0884  0.5161
         RMSE           0.0185  0.1362  0.0882  0.5149
168      MAE            0.0227  0.1508  0.0948  0.5706
         MSE            0.0204  0.1427  0.0950  0.5399
         RMSE           0.0205  0.1431  0.0946  0.5415

In [8]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.540612e+06  1239.7210   775.1422  0.0871
         MSE            1.623145e+06  1273.4576   831.4711  0.0895
         RMSE           1.654302e+06  1285.3571   832.0705  0.0903
96       MAE            3.377255e+06  1836.6797  1179.3198  0.1293
         MSE            3.399168e+06  1843.1373  1218.5483  0.1297
         RMSE           3.334171e+06  1825.9070  1205.1193  0.1285
168      MAE            4.200204e+06  2048.5980  1310.8441  0.1443
         MSE            4.229862e+06  2056.6512  1349.7440  0.1449
         RMSE           4.230690e+06  2056.8539  1345.3695  0.1449

# 4. MinMax Scaler (0, 1) PatchTST

In [21]:
log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [22]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.00001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0

In [23]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_relu_IT.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_relu_IT.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0114  0.1067  0.0656  0.4030
              2         24        0.0115  0.1074  0.0653  0.4058
              1         96        0.0182  0.1349  0.0871  0.5099
              2         96        0.0181  0.1345  0.0868  0.5084
              1         168       0.0194  0.1392  0.0916  0.5265
              2         168       0.0194  0.1392  0.0913  0.5267
RMSE          1         24        0.0114  0.1068  0.0658  0.4034
              2         24        0.0115  0.1071  0.0665  0.4048
              1         96        0.0181  0.1346  0.0867  0.5091
              2         96        0.0180  0.1342  0.0863  0.5074
              1         168       0.0193  0.1389  0.0912  0.5255
              2         168       0.0193  0.1388  0.0907  0.5252
MAE           1         24        0.0103  0.1014  0.0584  0.3833
              2         24        0.0106  0.1030  0.0597  0.3892
              1         96        0.0214  0.1464  0.0866  0.5535
              2         96        0.0203  0.1424  0.0852  0.5384
              1         168       0.0198  0.1406  0.0900  0.5321
              2         168       0.0198  0.1407  0.0900  0.5325

In [24]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1649581.250  1284.3602   850.4399  0.0903
              2         24        1637449.875  1279.6288   845.5874  0.0899
              1         96        3181651.500  1783.7185  1182.0157  0.1255
              2         96        3139555.500  1771.8790  1171.6736  0.1247
              1         168       3698332.250  1923.1049  1273.1566  0.1355
              2         168       3587113.500  1893.9677  1255.8635  0.1334
RMSE          1         24        1851359.875  1360.6469   874.3544  0.0956
              2         24        1904325.375  1379.9730   891.1001  0.0970
              1         96        3113013.750  1764.3734  1168.9977  0.1242
              2         96        3066430.250  1751.1226  1157.3804  0.1232
              1         168       3630107.250  1905.2841  1260.4178  0.1342
              2         168       3512351.750  1874.1270  1241.5947  0.1320
MAE           1         24        1229446.000  1108.8038   701.6744  0.0779
              2         24        1335150.250  1155.4871   726.6998  0.0812
              1         96        2624052.500  1619.8927  1041.3607  0.1140
              2         96        2622210.250  1619.3240  1035.5718  0.1140
              1         168       3422754.750  1850.0688  1207.7421  0.1303
              2         168       3396051.750  1842.8380  1203.2321  0.1298

In [25]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_0_1_relu.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0104  0.1022  0.0591  0.3862
         MSE            0.0115  0.1070  0.0655  0.4044
         RMSE           0.0114  0.1069  0.0661  0.4041
96       MAE            0.0209  0.1444  0.0859  0.5460
         MSE            0.0181  0.1347  0.0870  0.5092
         RMSE           0.0181  0.1344  0.0865  0.5083
168      MAE            0.0198  0.1407  0.0900  0.5323
         MSE            0.0194  0.1392  0.0915  0.5266
         RMSE           0.0193  0.1389  0.0910  0.5254

In [26]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.282298e+06  1132.1454   714.1871  0.0796
         MSE            1.643516e+06  1281.9945   848.0136  0.0901
         RMSE           1.877843e+06  1370.3099   882.7272  0.0963
96       MAE            2.623131e+06  1619.6083  1038.4662  0.1140
         MSE            3.160604e+06  1777.7988  1176.8447  0.1251
         RMSE           3.089722e+06  1757.7480  1163.1890  0.1237
168      MAE            3.409403e+06  1846.4534  1205.4871  0.1301
         MSE            3.642723e+06  1908.5363  1264.5101  0.1344
         RMSE           3.571230e+06  1889.7055  1251.0063  0.1331

In [27]:
# Rename folders
new_path_name = 'minmax_IT'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)